<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [257]:
import pandas as pd
import psycopg2

In [ ]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

In [258]:
# реальные параметры отключения заменены на пустой шаблон
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

In [259]:
# применим, чтобы отображался результат без лишней информации
import warnings
warnings.filterwarnings('ignore')

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [260]:
# текст запроса
query_3_1 = f''' SELECT
                     COUNT(id) AS vacancies_count
                 FROM VACANCIES
             '''
# использовала запрос на определение количества строк в столбце id в таблице VACANCIES

In [261]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df


,vacancies_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [262]:
# текст запроса
query_3_2 = f''' SELECT
                     COUNT(id) AS employers_count
                 FROM EMPLOYERS
             '''
# использовала запрос на определение количества строк в столбце id в таблице EMPLOYERS        

In [263]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

,employers_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [264]:
# текст запроса
query_3_3 = f''' SELECT
                     COUNT(DISTINCT id) AS areas_count
                 FROM AREAS
             '''
# использовала запрос на определение количества уникальных строк в столбце id в таблице AREAS 


In [265]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

,areas_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [266]:
# текст запроса
query_3_4 = f''' SELECT
                     COUNT(DISTINCT id) AS industries_count
                 FROM INDUSTRIES
             '''
# использовала запрос на определение количества уникальных строк в столбце id в таблице INDUSTRIES 

In [267]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

,industries_count
0,294


***

In [ ]:
# выводы по предварительному анализу данных

По результатам предварительного анализа данных можно сказать, что в нашей базе данных находится 49197 вакансий, 23501 работодателей. В среднем на каждого работодателя приходится 2.1 вакансий. Наша база охватывает 1362 региона и 294 сферы деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [268]:
# текст запроса
query_4_1 = f''' SELECT
                     COUNT(v.id) AS count_vacancies,
                     a.name AS name_areas
                 FROM VACANCIES v
                 JOIN AREAS a ON a.id = v.area_id
                 GROUP BY name_areas
                 ORDER BY count_vacancies DESC
             '''
# для определения количество вакансий использовала запрос на количества строк в столбце id таблицы VACANCIES
# таблицы VACANCIES и AREAS объединила методом JOIN
# чтобы получить число вакансий в каждом регионе, сделала группировку по столбцу name из таблицы AREAS
# для отсортировки по убыванию использовала созданный мной столбец count_vacancies

In [269]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,count_vacancies,name_areas
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
...,...,...
764,1,Тарко-Сале
765,1,Новоаннинский
766,1,Бирск
767,1,Сасово


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [270]:
# текст запроса
query_4_2 = f''' SELECT
                     COUNT(v.id) AS number_vacancies
                 FROM VACANCIES v
                 WHERE salary_from IS NOT NULL
                 OR salary_to IS NOT NULL
             '''
# для определения количество вакансий использовала запрос на определение количества строк в столбце id в таблице VACANCIES
# чтобы получить число вакансий с заполненной хотя бы одной из полей с зарплатой, применила условие, 
# что salary_from или salary_to не являются пустыми

In [271]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,number_vacancies
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [272]:
# текст запроса
query_4_3 = f''' SELECT
                     round(AVG(salary_from)) AS lower_salary_limit,
                     round(AVG(salary_to)) AS upper_salary_limit
                 FROM VACANCIES 
            '''
# использовала запрос для определения среднего значения нижней и верхней таблицы использую столбцы salary_from и salary_to соответственно из таб. VACANCIES 
# для округления применила round

In [273]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,lower_salary_limit,upper_salary_limit
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [274]:
# текст запроса
query_4_4 = f''' SELECT
                     COUNT(id) AS vacancies_number,
                     schedule,
                    employment
                 FROM VACANCIES
                 GROUP BY schedule, employment
                 ORDER BY vacancies_number DESC
             '''
# использовала запрос для определения количества вакансий использую столбец id из таб. VACANCIES 
# для определения количества вакансий для сочетаний типа рабочего графика и типа трудоустройства группировала по столбцам schedule, employment 
# отсортировала запрос по убыванию используя созданный в запросе столбец vacancies_number

In [275]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,vacancies_number,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [276]:
# текст запроса
query_4_5 = f''' SELECT
                     experience,
                     COUNT(id) AS count_vacancies
                 FROM VACANCIES
                 GROUP BY experience
                 ORDER BY count_vacancies

             '''
# использовала запрос для определения количества вакансий использую столбец id из таб. VACANCIES 
# для определения количества вакансий для каждого требуемого опыта работы группировала по столбцам experience
# отсортировала запрос по возрастанию используя созданный в запросе столбец count_vacancies

In [277]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,experience,count_vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [ ]:
# выводы по детальному анализу вакансий

- Из данных видно, что больше всего вакансий в крупных городах Москва, Санкт-Петербург, Новосибирск. Это связано с тем, что в крупных городах, как правило, больше рабочих мест, а так же много потенциальных работников. Еще крупные города, обычно, быстрее развиваются, и в них появляются новые рабочие места. Так же много вакансий предлагают крупные города из других стран Минск (Беларусь), Алматы(Казахстан). Возможно, это связано с тем, что наши работники ценятся как специалисты, а так же наш язык понимает местное население.
- По результатам детального анализа вакансий можно сказать, что только у 24073 из 49197 вакансий заполнено хотя бы одно из двух полей с зарплатой. Это значит, меньше чем 50% потенциальный работник знают на какую примерно зарплату они могут рассчитывать. Данный результат не очень хороший. Стоит объяснить работодателям, что нужно заполнять информацию о зарплате. Таким образом, работнику будет проще определиться с вакансией, а к работодателю будут приходить люди, которые готовы работать за указанную им зарплату.
- Средние значения для нижней и верхней границы зарплатной вилки равны 71065 и 110537 соответственно. Представленные данные не несут большой информативности, так как для каждой профессии и региона будут свои границы вилки. В добавок, больше чем у половины вакансий не указана зарплата, что так же снижает полезность данного запроса.
- Из данных можно сделать вывод, что самое популярное сочетания типа рабочего графика и типа трудоустройства это "Полный день — Полная занятость", количество вакансий равно 35367. На втором месте идет "Удалённая работа — Полная занятость" 7802 вакансии. На третьем месте "Гибкий график — Полная занятость" 1593 вакансии. Видно, что большинство работодателей ищут работников на полную занятость. На данный момент, появилось много вакансий на удаленную работу, но пока предпочтительней работа в "офисе".
- По данным видно, что больше всего работодатели ищут себе специалистов с опытом от 1 до 3, и от 3 до 6 лет, вакансий предлагают 26152 и 14511 соответственно. Это связано с желанием взять себе на работу человека со сформировавшимися профессиональными навыками. Не нужен опыта в начале для 7197 вакансий. Меньше всего вакансий для людей с опытом более 6 лет. Скорее всего, это связанно с тем, что такой большой опыт нужен не для всех должностей. Но человек с таким опытом спокойно найдет себе работу и среди вакансий, где нужно меньшее количество лет.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [278]:
# текст запроса
query_5_1 = f'''SELECT
                     e.name AS name_employers,
                     COUNT(v.id) AS vacancies_count
                 FROM VACANCIES v
                 JOIN EMPLOYERS e ON e.id = v.employer_id
                 GROUP BY name_employers
                 ORDER BY vacancies_count DESC
                 LIMIT 5
             '''
# использовала запрос для определения количества вакансий использую столбец id из таб. VACANCIES 
# таблицы VACANCIES и EMPLOYERS объединила методом JOIN
# для определения количества вакансий для каждого работодателя группировала по столбцу name_employers
# отсортировала запрос по убыванию используя созданный в запросе столбец vacancies_count 
# использовала лимит при отображении результата равны 5

In [279]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,name_employers,vacancies_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [280]:
# текст запроса
query_5_2 = f''' SELECT
                     a.name AS name_areas,
                    COUNT(distinct e.id) AS employers_count,
                    COUNT(distinct v.id) AS vakancies_count
                 FROM AREAS a
                 LEFT JOIN  EMPLOYERS e ON a.id = e.area
                 LEFT JOIN VACANCIES v ON a.id = v.area_id
                 WHERE v.id IS null
                 AND e.id IS NOT NULL
                 GROUP BY name_areas
                 ORDER BY employers_count DESC
            '''
# использовала запрос для определения количества работодателей и вакансий используя столбец id из таб.EMPLOYERS и столбец id из таб. VACANCIES соответственно
# таблицы AREAS, EMPLOYERS, VACANCIES объединила методом LEFT JOIN
# применила условие, что вакансий равны пустому значению, а работодатели не должны быть равны пустому значению
# для определения количества работодателей и вакансий для каждого региона группировала по столбцу name_areas
# отсортировала запрос по убыванию используя созданный в запросе столбец  employers_count 

In [281]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

,name_areas,employers_count,vakancies_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
181,Миллерово,1,0
182,"Михайловка (Уфимский район, Башкортостан)",1,0
183,Можайск,1,0
184,Морозовск\n,1,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [282]:
# текст запроса
query_5_3 = f''' SELECT
                     e.name AS employers_name,
                     COUNT(distinct v.area_id) AS count_area
                 FROM EMPLOYERS e
                 JOIN VACANCIES v ON e.id = employer_id
                 GROUP BY employers_name
                 ORDER BY count_area DESC
            '''
# использовала запрос для определения количества уникальных регионов, в которых работодатель публикует вакансии используя столбец area_id из таб.VACANCIES
# таблицы EMPLOYERS и VACANCIES объединила методом JOIN
# для определения количества уникальных регионов для каждого работодателя группировала по созданному столбцу employers_name
# отсортировала запрос по убыванию используя созданный в запросе столбец count_area

In [283]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

,employers_name,count_area
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [284]:
# текст запроса
query_5_4 = f''' SELECT
                     COUNT(e.id) AS count_employers
                 FROM EMPLOYERS e
                 LEFT JOIN EMPLOYERS_INDUSTRIES ei ON e.id = ei.employer_id
                 WHERE ei.industry_id IS NULL
             '''
# использовала запрос для определения количества работодателей используя столбец id из таб.EMPLOYERS
# таблицы EMPLOYERS и EMPLOYERS_INDUSTRIES объединила методом LEFT JOIN
# применила условие, что сфера деятельности должны быть равны пустому значению

In [285]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

,count_employers
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [286]:
# текст запроса
query_5_5 = f''' SELECT
                     e.name AS employers_name
                 FROM EMPLOYERS e
                 LEFT JOIN EMPLOYERS_INDUSTRIES ei ON e.id = ei.employer_id
                 GROUP BY employers_name
                 HAVING COUNT(ei.employer_id) = 4
                 ORDER BY employers_name
                 OFFSET 2 LIMIT 1
             '''
# использовала запрос для определения названия компании используя столбец name из таб.EMPLOYERS
# таблицы EMPLOYERS и EMPLOYERS_INDUSTRIES объединила методом LEFT JOIN 
# отфильтровала результат по тому, чтобы количество указанных сфер из столбца employer_id таб. EMPLOYERS_INDUSTRIES деятельности было равно 4
# группировала по созданному столбцу employers_name
# отсортировала запрос по алфавитному порядку используя созданный в запросе столбец employers_name 
# вырезала 2 первые строки и указала лимит на отображение 1

In [287]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

,employers_name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [288]:
# текст запроса
query_5_6 = f''' SELECT
                     COUNT(ei.employer_id) AS count_employer
                 FROM EMPLOYERS_INDUSTRIES ei
                 JOIN INDUSTRIES i ON ei.industry_id = i.id
                 WHERE i.name = 'Разработка программного обеспечения'
             '''
# использовала запрос для определения количества работодателей используя столбец employer_id из таб.EMPLOYERS_INDUSTRIES 
# таблицы EMPLOYERS_INDUSTRIES и INDUSTRIES объединила методом JOIN 
# отфильтровала результат по тому, чтобы в качестве сферы деятельности указано было «Разработка программного обеспечения».


In [289]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

,count_employer
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [192]:
# код для получения списка городов-милионников
#  парсинг таблицы из Wikipedia
import urllib.parse as req

path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
#  читаем ссылку
data = pd.read_html(url)[1]

In [195]:
#  данные  редактируем
city = list(data['Город']) # cтолбец переводим в список
city[13] = 'Воронеж' # меняем 13 элемент списка на верный 'Воронеж'
city = tuple(city) # переводим все в кортеж

In [290]:
# текст запроса
query_5_7 = f''' SELECT 
                     a.name, 
                     COUNT(v.id) AS count_vacancies_yandex
                     
                FROM VACANCIES v 
                JOIN AREAS a on v.area_id = a.id
                JOIN EMPLOYERS e on v.employer_id = e.id
                WHERE e.name = 'Яндекс' 
                AND a.name in {city}
                GROUP BY a.name
                UNION ALL
                SELECT 
                     'Total', 
                     COUNT(v.id) AS count_vacancies_yandex    
                FROM VACANCIES v 
                JOIN AREAS a on v.area_id = a.id
                JOIN EMPLOYERS e on v.employer_id = e.id
                WHERE e.name = 'Яндекс' 
                AND a.name in {city}
                ORDER BY count_vacancies_yandex
           '''
# использовала запрос для определения количества вакансия Яндекса используя столбец id из таб.VACANCIES
# таблицы VACANCIES, AREAS и EMPLOYERS объединила методом JOIN 
# отфильтровала результат по тому, чтобы в названии работодателя было указано Яндекс и название региона совпадало с названием регионов-миллионников  
# группировала по a.name 
# прибавила строку Total, отображаю общее количество вакансий Яндекса представленных в регионах-миллионников 
# отсортировала запрос по возрастанию используя созданный в запросе столбец count_vacancies_yandex

In [291]:
# результат запроса
df7 = pd.read_sql_query(query_5_7, connection)
df7

,name,count_vacancies_yandex
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

In [ ]:
# выводы по анализу работодателей

- По результатам анализа работодателей мы поняли, что больше всего вакансий предлагают компании Яндекс (1933), Ростелеком (491), Тинькофф (444), СБЕР (428), Газпром нефть (331). Можно сделать вывод, что в крупные компании требуется больше работников, что может быть связано с расширением или стандартной текучкой кадров, но в крупных компаниях из-за размера такая текучка больше.
- В качестве региона где нет вакансий, но есть работодатели, указывают чаще Россию. Мне кажется такая особенность появилась из-за того, что Россия это страна, а вакансии могут быть в нескольких городах. Поэтому при регистрации объявления компания может указать крупный объект, в который входят много объектов поменьше, а в вакансии указать уже конкретный город, в котором ищут человека. 
- В наибольшем количестве регионов опубликовали свои вакансии Яндекс (181 регион), Ростелеком (152), Спецремонт	(116). Скорей всего это связанно с тем, что у них есть вакансии на удаленку. Из-за чего они могут набирать работников из любого региона. Так же это может быть связано с распространенностью офисов в разных регионах.
- 8419 работодателей не указывают сфера деятельности. Это лучше исправить, так как многие работники могут хотеть работать в определенной сфере. И при поиске работы при фильтрации вакансий могут ее указать. Следовательно, они могут пропустить вакансию работодателя, даже если она им подходит, из-за того, что она не высветится в списке.
- 3553 работодателей в качестве сферы деятельности указали «Разработка программного обеспечения». В данный момент, все связанное с IT популярно. И указав данную сферу деятельности работодателю будет проще найти подходящего для своих нужд работника.
- Компания Яндекс, как одна из крупнейших компаний, представила вакансии в 16 крупных городах. Из них больше всего представлено в Москве (54) и Санкт-Петербурге(42). Всего в городах миллиониках представлено 485 вакансий из 1933. Это может быть связано с тем, что в Яндексе много вакансий подходящих для удаленной работы, и они ищут работников в разных регионах. Так Яндекс представлен в 181 регионе.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [292]:
# текст запроса
query_6_1 = f''' SELECT
                     COUNT(name) AS count_vacancies
                 FROM VACANCIES
                 WHERE name ILIKE '%data%'
                 OR name ILIKE '%ДАНН%'
             '''
# использовала запрос для определения количества вакансий используя столбец name из таб.VACANCIES
# отфильтровала результат по тому, чтобы где-нибудь в названии name содержались слова data, ДАНН, не учитывая регистр

In [293]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

,count_vacancies
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [294]:
# текст запроса
query_6_2 = f''' SELECT
                     COUNT(name) AS vacancies_count
                 FROM VACANCIES
                 WHERE ((name ILIKE '%data scientist%') 
                 OR (name ILIKE '%data science%')
                 OR(name ILIKE '%исследователь данных%') 
                 OR(name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                 OR(name ILIKE '%machine learning%')
                 OR(name ILIKE '%машинн%обучен%'))
                 AND (name ILIKE '%junior%'
                 OR employment = 'Стажировка'
                 OR experience = 'Нет опыта')
             '''
# использовала запрос для определения количества вакансий используя столбец name из таб.VACANCIES
# отфильтровала результат по тому, чтобы где-нибудь в названии name содержались слова data scientist, data science, исследователь данных, machine learning, машинн%обучен 
# не учитывая регистр, и учитывая регистр ML, а так же учитывая регистр не было HTML
# в условии фильтрации прописала так же, чтобы в названии было не учитывая регистр слово junior или employment равно Стажировка, или experience равно Нет опыта


In [295]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

,vacancies_count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [296]:
# текст запроса
query_6_3 = f''' SELECT
                     COUNT(name) AS count_vacancies_DS
                 FROM VACANCIES
                 WHERE ((name ILIKE '%data scientist%') 
                 OR (name ILIKE '%data science%')
                 OR(name ILIKE '%исследователь данных%') 
                 OR(name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                 OR(name ILIKE '%machine learning%')
                 OR(name ILIKE '%машинн%обучен%'))
                 AND (key_skills ILIKE '%SQL%'
                 OR key_skills ILIKE '%postgres%')
            '''
# использовала запрос для определения количества вакансий используя столбец name из таб.VACANCIES
# отфильтровала результат по тому, чтобы где-нибудь в названии name содержались слова data scientist, data science, исследователь данных, machine learning, машинн%обучен 
# не учитывая регистр, и учитывая регистр ML, а так же учитывая регистр не было HTML
# в условии фильтрации прописала так же, чтобы в key_skills где-нибудь не учитывая регистр было написано SQL или postgres

In [297]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

,count_vacancies_ds
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [298]:
# текст запроса
query_6_4 = f''' SELECT
                     COUNT(name) AS count_vacancies_python
                 FROM VACANCIES
                 WHERE ((name ILIKE '%data scientist%') 
                 OR (name ILIKE '%data science%')
                 OR(name ILIKE '%исследователь данных%') 
                 OR(name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                 OR(name ILIKE '%machine learning%')
                 OR(name ILIKE '%машинн%обучен%'))
                 AND key_skills ILIKE '%Python%'
            '''
# использовала запрос для определения количества вакансий используя столбец name из таб.VACANCIES
# отфильтровала результат по тому, чтобы где-нибудь в названии name содержались слова data scientist, data science, исследователь данных, machine learning, машинн%обучен 
# не учитывая регистр, и учитывая регистр ML, а так же учитывая регистр не было HTML
# в условии фильтрации прописала так же, чтобы в key_skills где-нибудь не учитывая регистр было написано Python

In [299]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

,count_vacancies_python
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [300]:
# текст запроса
query_6_5 = f''' SELECT
                     round(AVG(LENGTH(key_skills) - LENGTH(replace(key_skills,CHR(9),''))+1), 2) AS key_skills_DS
                FROM VACANCIES
                WHERE ((name ILIKE '%data scientist%') 
                OR (name ILIKE '%data science%')
                OR(name ILIKE '%исследователь данных%') 
                OR(name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                OR(name ILIKE '%machine learning%')
                OR(name ILIKE '%машинн%обучен%'))
                AND  key_skills IS NOT NULL

'''
# посчитала количество символов в строке в столбце key_skills и вычла количество символов в строке в столбце key_skills без табуляции
# к результату прибавила 1, чтобы получить число указанных навыков
# определила среднее количество навыков в столбец key_skills из таб.VACANCIES 
# результат округлили с помощью round до 2 знака после запятой
# отфильтровала результат по тому, чтобы где-нибудь в названии name содержались слова data scientist, data science, исследователь данных, machine learning, машинн%обучен 
# не учитывая регистр, и учитывая регистр ML, а так же учитывая регистр не было HTML
# в условии фильтрации прописала так же, чтобы в key_skills не был пустым

In [301]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

,key_skills_ds
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [302]:
# текст запроса
query_6_6 = f''' SELECT
                     experience,
                     round(AVG(coalesce((salary_from + salary_to)/2, salary_from, salary_to))) AS average_salary
                 FROM VACANCIES
                 WHERE ((name ILIKE '%data scientist%') 
                 OR (name ILIKE '%data science%')
                 OR(name ILIKE '%исследователь данных%') 
                 OR(name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                 OR(name ILIKE '%machine learning%')
                 OR(name ILIKE '%машинн%обучен%'))
                 AND (salary_from IS NOT NULL
                 OR salary_to IS NOT NULL)
                 GROUP BY experience
             '''
# посчитала среднею зарплату по столбцам salary_from и salary_to из таб.VACANCIES 
# результат округлили с помощью round
# отфильтровала результат по тому, чтобы где-нибудь в названии name содержались слова data scientist, data science, исследователь данных, machine learning, машинн%обучен 
# не учитывая регистр, и учитывая регистр ML, а так же учитывая регистр не было HTML
# в условии фильтрации прописала так же, чтобы salary_from или salary_to  небыли пустыми
# группировала по столбцу experience

In [303]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

,experience,average_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [45]:
# выводы по предметному анализу

- По результатам предметного анализа мы поняли, что 1771 вакансии имеют отношение к данным. Из них для начинающего дата-сайентиста подходит 51 вакансия.
- Из относящихся к данным для 201 вакансии в ключевые навыки входят SQL или postgres. Для 351 вакансии знания Python.
- В среднем 6.41 ключевых навыка указывают в вакансиях для DS. Средняя зарплата DS без опыта работы 74643 руб., с опытом зарплата растет. И уже с опытом от 3 до 6 года средняя зарплата равна 243115 рублей.
- Можно сказать, что DS очень важно обладать большим количеством навыков. Чем больше ты знаешь и умеешь, тем выше шансы устроиться на работу. Зарплаты в данной сфере хорошие.

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

- В данной базе  находится  49197 вакансий, 23501 работодателей,  1362 региона и 294 сферы деятельности. 
- Больше всего вакансий находятся в крупных городах Москва, Санкт-Петербург, Новосибирск. Так же есть много вакансии в крупных городах ближнего зарубежья. Только в 24073 из 49197 вакансий заполнено хотя бы одно из двух полей с зарплатой.  Стоит объяснить работодателям пользу информации о зарплате при поиске работника. Большинство работодателей ищут работников на полную занятость и полный день, таких вакансий 35367, что равно примерно 72 % от всех вакансий. Так же предпочтительней иметь опыт работы. Так в базе представлено  7197 вакансий не требующих опыта работы, что составляет примерно 15 % от всех вакансий. 
- Чем крупнее компания, тем больше работников они себе ищут. Больше всего вакансий выложили компании Яндекс (1933 вакансии), Ростелеком (491 вакансии), Тинькофф (444 вакансии), СБЕР (428 вакансии), Газпром нефть (331 вакансии). При этом в наибольшем количестве регионах представлены компании  Яндекс (181 регион), Ростелеком (152 региона), Спецремонт (116 региона). Что может быть связано с большой возможностью работать в них на удалённой работе, а так же их распространенностью в разных регионах. Видно, что самым популярным работодателем является Яндекс. Он предлагает наибольшее число вакансий, во многих регионах. Что связано с тем, что это крупный работодатель в сфере IT  в России и на него можно работать удаленно.  Сфера IT популярна, так 3553 работодателя в качестве сферы деятельности указали «Разработка программного обеспечения».
- Из всех вакансии 1771 имеют отношение к данным. Из них для начинающего дата-сайентиста подходит 51 вакансия. Очень важно иметь в ключевых навыках  знание SQL или postgres, а также Python. Чем большем количеством ключевых навыков обладает человек, тем проще ему будет найти работу. В среднем нужно иметь 6 - 7 ключевых навыков. Зарплаты в данной профессии хорошие и растут с опытом работы.

# 7. Дополнительные исследования данных

1. Среднее количество ключевых навыков для начинающиго специалиста junior в DS.

In [304]:
# текст запроса
query_7_1 = f'''SELECT
                     round(AVG(LENGTH(key_skills) - LENGTH(replace(key_skills,CHR(9),''))+1)) AS key_skills_junior
                FROM VACANCIES
                WHERE ((name ILIKE '%data scientist%') 
                OR (name ILIKE '%data science%')
                OR(name ILIKE '%исследователь данных%') 
                OR(name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                OR(name ILIKE '%machine learning%')
                OR(name ILIKE '%машинн%обучен%'))
                AND  key_skills IS NOT NULL
                AND (name ILIKE '%junior%'
                OR employment = 'Стажировка'
                OR experience = 'Нет опыта')
             '''
# посчитала количество символов в строке в столбце key_skills и вычла количество символов в строке в столбце key_skills без табуляции
# к результату прибавила 1, чтобы получить число указанных навыков
# определила среднее количество навыков в столбец key_skills из таб.VACANCIES 
# результат округлили с помощью round
# отфильтровала результат по тому, чтобы где-нибудь в названии name содержались слова data scientist, data science, исследователь данных, machine learning, машинн%обучен 
# не учитывая регистр, и учитывая регистр ML, а так же учитывая регистр не было HTML
# в условии фильтрации прописала так же чтобы в названии было не учитывая регистр слово junior или employment равно Стажировка, или experience равно Нет опыта

In [305]:
# результат запроса
df = pd.read_sql_query(query_7_1, connection)
df

,key_skills_junior
0,7.0


2. На какую минимальную зарплату может рассчитывать специалист в DS при каждом типе опыта работы.

In [306]:
# текст запроса
query_7_2 = f''' SELECT
                     experience,
                     round(MIN(coalesce((salary_from + salary_to) - salary_to, salary_from, salary_to))) AS average_salary_min
                 FROM VACANCIES
                 WHERE ((name ILIKE '%data scientist%') 
                 OR (name ILIKE '%data science%')
                 OR(name ILIKE '%исследователь данных%') 
                 OR(name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                 OR(name ILIKE '%machine learning%')
                 OR(name ILIKE '%машинн%обучен%'))
                 AND (salary_from IS NOT NULL
                 OR salary_to IS NOT NULL)
                 GROUP BY experience
             '''
# посчитала минимальную зарплату по столбцам salary_from и salary_to из таб.VACANCIES 
# результат округлили с помощью round
# отфильтровала результат по тому, чтобы где-нибудь в названии name содержались слова data scientist, data science, исследователь данных, machine learning, машинн%обучен 
# не учитывая регистр, и учитывая регистр ML, а так же учитывая регистр не было HTML
# в условии фильтрации прописала так же, чтобы salary_from или salary_to  небыли пустыми
# группировала по столбцу experience

In [307]:
# результат запроса
df = pd.read_sql_query(query_7_2, connection)
df

,experience,average_salary_min
0,Нет опыта,37000.0
1,От 1 года до 3 лет,50000.0
2,От 3 до 6 лет,61024.0


3. Определить количество вакансий DS для каждого сочетания типа рабочего графика и типа трудоустройства указанных в базе данных.

In [308]:
# текст запроса
query_7_3 = f''' SELECT
                     COUNT(id) AS vacancies_number_DS,
                     schedule,
                    employment
                 FROM VACANCIES
                 WHERE ((name ILIKE '%data scientist%') 
                 OR (name ILIKE '%data science%')
                 OR(name ILIKE '%исследователь данных%') 
                 OR(name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                 OR(name ILIKE '%machine learning%')
                 OR(name ILIKE '%машинн%обучен%'))
                 GROUP BY schedule, employment
                 ORDER BY vacancies_number_DS DESC
             '''
# использовала запрос для определения количества вакансий использую столбец id из таб. VACANCIES 
# отфильтровала результат по тому, чтобы где-нибудь в названии name содержались слова data scientist, data science, исследователь данных, machine learning, машинн%обучен 
# не учитывая регистр, и учитывая регистр ML, а так же учитывая регистр не было HTML
# для определения количества вакансий для сочетаний типа рабочего графика и типа трудоустройства группировала по столбцам schedule, employment 
# отсортировала запрос по убыванию используя созданный в запросе столбец vacancies_number_DS

In [309]:
# результат запроса
df = pd.read_sql_query(query_7_3, connection)
df

,vacancies_number_ds,schedule,employment
0,319,Полный день,Полная занятость
1,110,Удаленная работа,Полная занятость
2,37,Гибкий график,Полная занятость
3,4,Гибкий график,Частичная занятость
4,3,Удаленная работа,Частичная занятость
5,2,Полный день,Стажировка
6,2,Удаленная работа,Проектная работа
7,2,Гибкий график,Стажировка
8,1,Полный день,Частичная занятость


4. Сколько вакансий для DS предлагают разные работодатели.

In [310]:
# текст запроса
query_7_4 = f''' SELECT
                     COUNT(v.id) AS vacancies_count_DS,
                     e.name
                 FROM VACANCIES v
                 JOIN EMPLOYERS e on v.employer_id = e.id
                 WHERE ((v.name ILIKE '%data scientist%') 
                 OR (v.name ILIKE '%data science%')
                 OR(v.name ILIKE '%исследователь данных%') 
                 OR(v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
                 OR(v.name ILIKE '%machine learning%')
                 OR(v.name ILIKE '%машинн%обучен%'))
                 GROUP BY e.name
                 ORDER BY vacancies_count_DS DESC
             '''
# использовала запрос для определения количества вакансий использую столбец id из таб. VACANCIES 
# таблицы VACANCIES и EMPLOYERS объединила методом JOIN
# отфильтровала результат по тому, чтобы где-нибудь в названии name содержались слова data scientist, data science, исследователь данных, machine learning, машинн%обучен 
# не учитывая регистр, и учитывая регистр ML, а так же учитывая регистр не было HTML
# групировала по name из таб. EMPLOYERS
# отсортировала запрос по убыванию используя созданный в запросе столбец vacancies_count_DS DESC

In [311]:
# результат запроса
df = pd.read_sql_query(query_7_4, connection)
df

,vacancies_count_ds,name
0,37,СБЕР
1,25,Bell Integrator
2,18,Банк ВТБ (ПАО)
3,15,VK
4,11,Positive Technologies
...,...,...
220,1,Иллекс
221,1,ИНВИТРО
222,1,Институт искусственного интеллекта AIRI
223,1,Интеллектуальные решения


In [312]:
# закрытие соединения
connection.close()

Вывод по-дополнительному исследованию. 
- Для специалиста junio очень важно обладать большим количеством навыков. Так среднее количество навыков для данного специалиста 7. 
- При этом специалист без опыта работы может рассчитывать на минимальную зарплату в размере 37000 рублей. 
- Работодатели предпочитают брать работников на полную занятость. На первом месте стоит тип рабочего графика "Полный день" 319 вакансий, на втором месте "Удаленная работа" 110 вакансий. Работая в данной сфере можно спокойно найти работу на удаленную работу. 
- Больше всего вакансий для DS предлагают банки, так Сбер выложил 37 вакансий, а ВТБ 18. Компании относящиеся к другим услугам как Bell Integrator предложили 25, и VK 15. При этом 225 работодателя ищут DS. Видно, что данный специалист нужен во многих компаниях.
- В будущем интересно посмотреть на динамику роста или уменьшения популярности специальности DS. Так же интересно будет исследовать количество кандитатов на разные специальности, чтобы понять насколько конкурентная среда в каждой сфере.